In [1]:
%load_ext autoreload
%autoreload 2

import logging
import gradio as gr

from agent.agents import (
    create_multi_query_agent_executor, 
    create_one_action_agent_executor,
    stream_agent_output,
    StateCapture
)
from config.utils import setup_logging, get_logger
from config.config import ConfigManager

# checking where we are
!pwd

/Users/eliza/search-pydantic-ai/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/Users/eliza/search-pydantic-ai


## Config and logging

In [6]:
setup_logging(level=logging.INFO)
logger = get_logger(__name__)
logger.warning("Starting the application...")
logger.info("Starting the application...")
config = ConfigManager()

print(config.documentation_urls)
print(config.config['embed_with_openai'])

2025-05-14 17:02:38,047 - __main__ - WARNING - Starting the application...
2025-05-14 17:02:38,049 - __main__ - INFO - Starting the application...
2025-05-14 17:02:38,050 - config.config - INFO - Attempting to load configuration from config.json
2025-05-14 17:02:38,053 - config.config - INFO - Configuration loaded successfully.
2025-05-14 17:02:38,055 - config.config - INFO - Configuration validated.
['https://ai.pydantic.dev/']
True


## One action agent interface

**Scroll below for the interactive interface**

In [11]:
state_capture = StateCapture()
agent_executor = create_one_action_agent_executor()

In [12]:
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(height=320)
    # Text input with stop button on right
    with gr.Row():
        with gr.Column(scale=20):
            msg = gr.Textbox(label="Message", placeholder="Ask something...")
            
        with gr.Column(scale=1, min_width=40):
            stop_btn = gr.Button("⏹️", size="sm", variant="stop")
    
    # Buttons below the text input        
    with gr.Row():
        send_btn = gr.Button("Send", size="md", variant="primary", scale=3)
        clear = gr.Button("Clear", size="md", scale=1)
    
    def respond(message, chat_history):
        # Add user message to chat history
        chat_history.append((message, ""))
        yield chat_history
        
        # Stream the agent's response
        bot_message = ""
        for chunk in stream_agent_output(agent_executor, message, state_capture):
            bot_message += chunk
            # Update just the last bot message
            chat_history[-1] = (message, bot_message)
            yield chat_history
    
    # Set up events with cancellation
    msg_event = msg.submit(respond, [msg, chatbot], [chatbot])
    send_event = send_btn.click(respond, [msg, chatbot], [chatbot])
    
    # Clear button
    clear.click(lambda: None, None, chatbot, queue=False)
    
    # Stop button cancels both the message submit and send button events
    stop_btn.click(fn=None, inputs=None, outputs=None, cancels=[msg_event, send_event])

demo.launch(inline=True, share=False)

/var/folders/3n/7m69d59d44x4c5xxm3gwg9880000gn/T/ipykernel_93576/1855339524.py:2: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=320)


* Running on local URL:  http://127.0.0.1:7862
2025-05-14 17:08:54,389 - httpx - INFO - HTTP Request: GET http://127.0.0.1:7862/gradio_api/startup-events "HTTP/1.1 200 OK"
2025-05-14 17:08:54,399 - httpx - INFO - HTTP Request: HEAD http://127.0.0.1:7862/ "HTTP/1.1 200 OK"
* To create a public link, set `share=True` in `launch()`.


2025-05-14 17:08:54,957 - httpx - INFO - HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"


## Multiquery and follow up agent interface
**Scroll down for gradio interfaces**

In [13]:
import gradio as gr
from agent.agents import (
    create_multi_query_agent_executor, 
    stream_agent_output,
    StateCapture
)

state_capture = StateCapture()
agent_executor = create_multi_query_agent_executor()

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(height=320)
    # Text input with stop button on right
    with gr.Row():
        with gr.Column(scale=20):
            msg = gr.Textbox(label="Message", placeholder="Ask something...")
            
        with gr.Column(scale=1, min_width=40):
            stop_btn = gr.Button("⏹️", size="sm", variant="stop")
    
    # Buttons below the text input        
    with gr.Row():
        send_btn = gr.Button("Send", size="md", variant="primary", scale=3)
        clear = gr.Button("Clear", size="md", scale=1)
    
    def respond(message, chat_history):
        # Add user message to chat history
        chat_history.append((message, ""))
        yield chat_history
        
        # Message tracking variables
        bot_message = ""
        current_mode = "normal"  # Tracking if we're in a "searching" or "normal" mode
        searching_complete = False  # Track if we've seen a searching message
        
        # Stream the agent's response
        for chunk in stream_agent_output(agent_executor, message, state_capture):
            # Detect message type transitions
            if chunk.startswith("Searching:"):
                # We're in searching mode
                if current_mode == "normal" and bot_message:
                    # If we already had normal content, separate it
                    bot_message += "\n\n---\n\n"
                
                current_mode = "searching"
                searching_complete = True
                bot_message += chunk
                
            else:
                # We're in normal response mode
                if current_mode == "searching" and searching_complete:
                    # Transition from searching to normal - add separator
                    bot_message += "\n\n---\n\n"
                    current_mode = "normal"
                    
                # Add content to current message
                bot_message += chunk
                
            # Update chat history with formatted message
            chat_history[-1] = (message, bot_message)
            yield chat_history

    
    # Set up events with cancellation
    msg_event = msg.submit(respond, [msg, chatbot], [chatbot])
    send_event = send_btn.click(respond, [msg, chatbot], [chatbot])
    
    # Clear button
    clear.click(lambda: None, None, chatbot, queue=False)
    
    # Stop button cancels both the message submit and send button events
    stop_btn.click(fn=None, inputs=None, outputs=None, cancels=[msg_event, send_event])

demo.launch(inline=True, share=False)

/var/folders/3n/7m69d59d44x4c5xxm3gwg9880000gn/T/ipykernel_14339/1468504269.py:12: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=320)


* Running on local URL:  http://127.0.0.1:7868
* To create a public link, set `share=True` in `launch()`.


## Check evaluations

In [23]:
from langsmith import Client
import os
from datetime import datetime, timedelta
from dotenv import load_dotenv

load_dotenv()

client = Client(api_key=os.getenv("LANGSMITH_API_KEY"))
runs =client.list_runs(
    project_name=os.getenv("LANGSMITH_PROJECT"),
    run_type="chain",
    start_time=datetime.now() - timedelta(days=1),
)

hallucination_scores = []
runs_with_feedback = [run for run in runs if run.feedback_stats]
for run in runs_with_feedback:
    feedbacks = client.list_feedback(run_ids=[run.id])
    hallucination_score = [feedback.score for feedback in feedbacks if feedback.key == "hallucination_score"]
    hallucination_scores.append(hallucination_score)

hallucination_scores = [score[0] for score in hallucination_scores if score]





In [25]:
from numpy import mean, std

print(f"Mean hallucination score: {mean(hallucination_scores)}")
print(f"Standard deviation of hallucination scores: {std(hallucination_scores)}")



Mean hallucination score: 0.08536585365853659
Standard deviation of hallucination scores: 0.21815297341461365
